# ELI5 Dataset - DeepSeek Answer Generation

This notebook processes the ELI5 (Explain Like I'm 5) dataset and generates LLM answers using DeepSeek.

## Dataset Information
- **Source**: HuggingFace dataset `rexarski/eli5_category`
- **Period**: January 2017 - June 2021
- **Content**: Human-written questions and answers from the ELI5 subreddit
- **Purpose**: Generate one LLM answer per unique question and merge with human responses

####  **<span style="color:red">IMPORTANT: <span>**

**Workflow & Integration:**
1. This notebook: Load dataset → Test API → Generate DeepSeek answers → Merge with human answers
2. Model: deepseek-chat
3. Each unique question gets ONE LLM answer replicated across all human responses
4. **To combine with other models:**
   - Run `openai_generate_dataset.ipynb` first (generates human + chatgpt answers)
   - Run `gemini_generate_dataset.ipynb` (generates gemini answers)
   - Run this notebook (generates deepseek answers)
   - Use the merge function to combine all without duplicating human answers
   - Final dataset: human + chatgpt + gemini + deepseek sources

## 1. Install and Import Required Libraries

In [ ]:
# Install required packages (run once)
# !pip install pandas numpy datasets
# !pip install openai
# !pip install gdown matplotlib seaborn tqdm
# !pip install fastparquet

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import json
import os
import glob
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

# OpenAI-compatible API (for DeepSeek)
from openai import OpenAI

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

## 2. Set Up DeepSeek API Key

In [2]:
import dotenv
dotenv.load_dotenv()
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

if DEEPSEEK_API_KEY:
    client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")
    print("DeepSeek API key configured successfully")
else:
    print("DeepSeek API key not set")

DeepSeek API key configured successfully


## 3. Load the ELI5 Dataset

## Configuration - Set Parameters Here

In [3]:
num_questions_to_generate = 5000
delay_between_api_calls = 1
test_sample_size = 2
max_workers = 6  # Number of concurrent API calls

deepseek_model = "deepseek-chat"

print("Configuration:")
print(f"  Questions to generate: {num_questions_to_generate}")
print(f"  Delay between calls: {delay_between_api_calls}s")
print(f"  Max concurrent workers: {max_workers}")
print(f"  Test sample size: {test_sample_size}")
print(f"  Model: {deepseek_model}")

Configuration:
  Questions to generate: 5000
  Delay between calls: 1s
  Max concurrent workers: 6
  Test sample size: 2
  Model: deepseek-chat


In [4]:
path = "./human_data/output/eli5_cleaned.csv"

df_human = pd.read_csv(path)
print(f"Dataset loaded with {len(df_human)} records")

# Get unique questions
unique_questions = df_human[['q_id', 'title']].drop_duplicates().reset_index(drop=True)
print(f"Found {len(unique_questions)} unique questions")
print(f"Average answers per question: {len(df_human) / len(unique_questions):.1f}")

Dataset loaded with 228637 records
Found 93893 unique questions
Average answers per question: 2.4


In [5]:
# Display first few rows
print("\nFirst few rows of dataset:")
print("=" * 80)
df_human[['q_id', 'title', 'text']].head(3)


First few rows of dataset:


,q_id,title,text
0,5lchat,Why there was a 'leap second' added to the end of 2016?,"the rotation of the earth is not a constant. in fact the rotation of the earth is slowing down, which means that a full day is getting slightly longer. without leap seconds our clocks would slowly drift ever so slightly out of sync with the actual day. we could deal with this by redefining how how long 1 second is, making it slightly longer so that one day is still exactly 24*60*60 seconds. but in practice that is really inconvenient for a lot of our technology which relies on very precise timing. its easier to just move us ahead one second every couple of years or so."
1,5lchat,Why there was a 'leap second' added to the end of 2016?,"The Earth's rotation is not regular. It varies a bit, so sometimes we add a second. We do this to ensure that noon is always going to be sometime around mid-day. If we did not add leap seconds, over a very long period of time where the Earth's rotation slowly changed, noon could end up being at dusk. We want to keep 7am in the morning, noon at mid-day, 7pm around evening, etc. Though we have never had one, it's also possible to have a negative leap second. That is, taking away a second from the year. This has never happened, but if the Earth's rotation were to speed up, it could happen. The biggest thing to know about leap seconds is that they can cause computer problems. You might remember the Y2K bug. A leap second can cause similar problems, and they actually have caused problems in the past. The reason for this is that generally we expect a day to have 24 hours, and for time to always move forward. With a leap second this is not true. When writing software, programers try to think of all the possible exceptions that could happen withing their code. For example, the program might expect a word, but instead get a number. A good programmer will check for these exceptions and deal with them. However, a programer can easily forget about leap seconds and not have a fail-safe in their code for when a day have more than 24 hours. When such an exception happens, the program can produce errors or crash. It is an interesting topic, you can read more about it here: URL_0"
2,5lchat,Why there was a 'leap second' added to the end of 2016?,"Because the Earth's rotation is slowing. If you multiply 24 hours by 60 minutes by 60 seconds, you find that there are 86400 seconds per day. The problem is that our definition of the second is based on [an average that is a century old.] In modern times, the average day is about 2 thousandths of a second longer—again, because of Earth's slowing rotation. Those thousandths of a second add up, so every few years we have to slip in an extra second to account for them. Without leap seconds, we'd eventually end up with noon at 7 o'clock, though admittedly, this would take a very long time."


In [8]:
# Load existing generated answers from output folder to avoid regenerating
output_folder = "deepseek-output"
existing_q_ids = set()

if os.path.exists(output_folder):
    # Find all CSV files in the output folder
    csv_files = [f for f in os.listdir(output_folder) if f.startswith('eli5_deepseek_answers_') and f.endswith('.csv')]
    
    if csv_files:
        print(f"Found {len(csv_files)} existing output file(s):")
        for csv_file in csv_files:
            filepath = os.path.join(output_folder, csv_file)
            try:
                df_existing = pd.read_csv(filepath)
                existing_q_ids.update(df_existing['q_id'].unique())
                print(f"  - {csv_file}: {df_existing['q_id'].nunique()} questions")
            except Exception as e:
                print(f"  - {csv_file}: Error reading file - {str(e)}")
        
        print(f"\nTotal unique questions already generated: {len(existing_q_ids)}")
    else:
        print("No existing output files found")
else:
    print(f"Output folder '{output_folder}' does not exist - starting fresh")

# Filter out already-processed questions
unique_questions_filtered = unique_questions[~unique_questions['q_id'].isin(existing_q_ids)].reset_index(drop=True)

print(f"Total unique questions: {len(unique_questions)}")
print(f"Already generated: {len(existing_q_ids)}")
print(f"Remaining to generate: {len(unique_questions_filtered)}")

if len(unique_questions_filtered) == 0:
    print("\nAll questions have already been processed")

Output folder 'deepseek-output' does not exist - starting fresh
Total unique questions: 93893
Already generated: 0
Remaining to generate: 93893


## 4. Define LLM Answer Generation Function

In [9]:
def generate_deepseek_answer(question, model="deepseek-chat", max_retries=3):
    """
    Generate an ELI5-style answer using DeepSeek.
    
    Args:
        question: The question to answer
        model: DeepSeek model to use (default: deepseek-chat)
        max_retries: Number of retry attempts on failure
    
    Returns:
        Generated answer as string, or error message if failed
    """
    if not DEEPSEEK_API_KEY:
        return "ERROR: DeepSeek API key not configured"
    
    system_prompt = """You are answering questions in the style of the ELI5 (Explain Like I'm 5) subreddit. 
Provide a clear, simple explanation that a 5-year-old could understand, but still be informative.
Keep everything as one block of text."""
    
    user_prompt = f"Question: {question}\n\nAnswer:"
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.7,
                max_tokens=1000,
            )
            return response.choices[0].message.content.strip()
        
        except Exception as e:
            print(f"  Attempt {attempt + 1} failed: {str(e)[:100]}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                return f"ERROR: {str(e)}"
    
    return None

## 5. Test API with Sample Questions

In [10]:
# Test with sample questions
print(f"Testing API with {test_sample_size} sample questions...")
print("=" * 80)

test_questions = unique_questions.head(test_sample_size)

for idx, row in test_questions.iterrows():
    question = row['title']
    print(f"\n[Test {idx + 1}] {question[:70]}...")
    answer = generate_deepseek_answer(question)
    
    if answer.startswith("ERROR"):
        print(f"{answer}")
    else:
        print(f"Generated ({len(answer)} chars): {answer[:100]}...")
    
    time.sleep(1)

print("API test complete!")

Testing API with 2 sample questions...

[Test 1] Why there was a 'leap second' added to the end of 2016?...
Generated (641 chars): Think of Earth like a spinning top that's starting to wobble and slow down just a tiny bit over a ve...

[Test 2] How do you claim undiscovered land?...
Generated (337 chars): You can't just go and claim any empty land you find because most land already belongs to a country, ...
API test complete!


## 6. Generate LLM Answer for Each Unique Question

In [11]:
# Generate one LLM answer per unique question using parallel processing
llm_answers_map = {}  # Map from q_id to llm_answer

questions_to_process = unique_questions_filtered.head(min(num_questions_to_generate, len(unique_questions_filtered)))

def process_single_question(row):
    """
    Process a single question and return (q_id, answer) tuple.
    """
    q_id = row['q_id']
    question = row['title']
    
    try:
        llm_answer = generate_deepseek_answer(question)
        
        if not llm_answer.startswith("ERROR"):
            return (q_id, llm_answer, True)
        else:
            return (q_id, llm_answer[:80], False)
    except Exception as e:
        return (q_id, f"ERROR: {str(e)[:80]}", False)

# Check if there are questions to process
if len(questions_to_process) == 0:
    print("No questions to process. All questions have been generated already.")
else:
    print(f"Generating {len(questions_to_process)} LLM answers with {max_workers} concurrent workers...")
    print("=" * 80)
    
    # Use ThreadPoolExecutor for concurrent API calls
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        futures = {executor.submit(process_single_question, row): idx 
                   for idx, row in questions_to_process.iterrows()}
        
        # Process completed futures with progress bar
        with tqdm(total=len(futures), desc="Generating answers") as pbar:
            for future in as_completed(futures):
                q_id, result, success = future.result()
                
                if success:
                    llm_answers_map[q_id] = result
                else:
                    print(f"\nSkipped {q_id}: {result}")
                
                pbar.update(1)
                
                # Rate limiting: brief delay after each completion
                time.sleep(delay_between_api_calls / max_workers)

    print(f"\n{'=' * 80}")
    print(f"Generated {len(llm_answers_map)} / {len(questions_to_process)} LLM answers")

Generating 5000 LLM answers with 6 concurrent workers...


Generating answers: 100%|██████████| 5000/5000 [39:44<00:00,  2.10it/s] 


Generated 5000 / 5000 LLM answers


## 7. Create LLM Answers Dataset

In [12]:
# Create LLM answers dataset
llm_rows_list = []

for q_id, llm_answer in llm_answers_map.items():
    # Get the question info
    question_info = unique_questions[unique_questions['q_id'] == q_id].iloc[0]
    
    llm_rows_list.append({
        'q_id': q_id,
        'title': question_info['title'],
        'text': llm_answer,
        'source': 'deepseek'
    })

df_llm = pd.DataFrame(llm_rows_list).reset_index(drop=True)

print(f"Total new rows: {len(df_llm)}")
print(f"Unique questions: {df_llm['q_id'].nunique()}")
print(f"Columns: {list(df_llm.columns)}")

if len(df_llm) == 0:
    print("\nNo new answers generated in this run.")

Total new rows: 5000
Unique questions: 5000
Columns: ['q_id', 'title', 'text', 'source']


In [13]:
# Show sample rows
print("\nSample LLM answers:")
print("=" * 80)

for idx, row in df_llm.head(3).iterrows():
    print(f"\n[{idx + 1}] Question: {row['title'][:70]}...")
    print(f"    Answer: {row['text'][:100]}...")
    print(f"    Source: {row['source']}")


Sample LLM answers:

[1] Question: How do you claim undiscovered land?...
    Answer: You can't just go and claim any empty land you find because most of the world is already owned by co...
    Source: deepseek

[2] Question: Why do we fail to do realistic human CGI (like in SW Rouge One) yet we...
    Answer: We can make amazing CGI animals and monsters because our brains aren't as picky about how they shoul...
    Source: deepseek

[3] Question: Why is it that we calm down when we take a deep breath, hold it for a ...
    Answer: When you feel scared or upset, your body gets ready to run or fight, which makes your heart beat fas...
    Source: deepseek


## 8. Save LLM Answers Dataset

In [14]:
# Create output folder
output_folder = "deepseek-output"
os.makedirs(output_folder, exist_ok=True)

# Only save if there are new answers generated
if len(df_llm) > 0:
    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Save to CSV
    csv_filename = os.path.join(output_folder, f"eli5_deepseek_answers_{timestamp}.csv")
    df_llm.to_csv(csv_filename, index=False)
    print(f"CSV saved to: {csv_filename}")

    # Save as Parquet
    parquet_filename = os.path.join(output_folder, f"eli5_deepseek_answers_{timestamp}.parquet")
    df_llm.to_parquet(parquet_filename, index=False)
    print(f"Parquet saved to: {parquet_filename}")

    # Save metadata
    metadata = {
        'timestamp': timestamp,
        'total_rows': int(len(df_llm)),
        'unique_questions': int(df_llm['q_id'].nunique()),
        'deepseek_answers': int(len(df_llm)),
        'llm_model': deepseek_model,
        'max_workers': max_workers,
        'columns': list(df_llm.columns)
    }

    metadata_filename = os.path.join(output_folder, f"metadata_{timestamp}.json")
    with open(metadata_filename, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"Metadata saved to: {metadata_filename}")
    print(f"Generated {len(df_llm)} new LLM answers")
    print(f"Output folder: {output_folder}")
else:
    print("All questions have already been processed in previous runs.")

CSV saved to: deepseek-output\eli5_deepseek_answers_20260217_003444.csv
Parquet saved to: deepseek-output\eli5_deepseek_answers_20260217_003444.parquet
Metadata saved to: deepseek-output\metadata_20260217_003444.json
Generated 5000 new LLM answers
Output folder: deepseek-output


## 9. Merge all generated answers

In [15]:
# Merge all CSVs under deepseek folder
output_folder = "deepseek-output"

# Find all CSV files in the output folder
csv_pattern = os.path.join(output_folder, "eli5_deepseek_answers_*.csv")
csv_files = glob.glob(csv_pattern)

if not csv_files:
    print(f"No CSV files found in {output_folder}")
else:
    for f in csv_files:
        print(f"  - {os.path.basename(f)}")
    
    # Load and concatenate all CSV files
    dfs = []
    for csv_file in csv_files:
        try:
            df = pd.read_csv(csv_file)
            dfs.append(df)
        except Exception as e:
            print(f"Error loading {csv_file}: {str(e)}")
    
    if dfs:
        # Merge all dataframes
        df_merged = pd.concat(dfs, ignore_index=True)
        
        # Remove duplicates based on q_id (keep first occurrence)
        df_merged_unique = df_merged.drop_duplicates(subset=['q_id'], keep='first')
       
        # Save merged file
        merged_filename = os.path.join(output_folder, "eli5_deepseek_answers_merged.csv")
        df_merged_unique.to_csv(merged_filename, index=False)
        
        # Save as parquet
        merged_parquet = os.path.join(output_folder, "eli5_deepseek_answers_merged.parquet")
        df_merged_unique.to_parquet(merged_parquet, index=False)
    else:
        print("No data loaded to merge")

  - eli5_deepseek_answers_20260217_003444.csv
